<a href="https://colab.research.google.com/github/pcpiscator/Ciencia-de-dados/blob/master/Titanic01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio "Kaggle": Titanic

Este desafio foi preparado e disponibilizado pelo [Kaggle](https://www.kaggle.com/c/titanic), um website adquirido pelo Google para promover competições na área de ciência de dados e aprendizado de máquina. 

O **Titanic** é um dos eventos mais tristes da história de naugrágios. 

Em 15 de abril de 1912, durante sua viagem inaugural, o então considerado navio de passageiros com zero chance de naufrágio RMS Titanic afundou após colidir com um *iceberg*. Infelizmente, o RMS Titanic não contava com a quantidade suficiente de botes salva-vidas para todos a bordo, resultando com isto, na morte de 1502 passageiros de um total de uma tripulação de 2224 pessoas. 

Apesar de existir um elemento de sorte envolvida na sobrevivência neste desastre, parece que alguns grupos de pessoas tiveram mais chances de sobrevivências que outros. 

***Neste desafio, pede-se a você que construa um modelo preditivo que traga uma resposta para a pergunta: "quais grupos de pessoas tiveram mais chances de sobreviver neste desastre"? Para tanto, você deve explorar os dados de passageiros (ex. nome, idade, gênero, classe sócio-econômica, e outros).*** 

Este notebook apresenta um passo-a-passo na análise exploratória dos dados de passageiros, iniciando com um entendimento, logo após tratamento e por último, chegando na modelagem para responder a pergunta acima. 



<img src="https://19emhilsa.weebly.com/uploads/4/5/1/4/45147515/451019_orig.jpg"> </img>





# Previsão de Sobreviventes no Naufrágio do Titanic

### Conteúdo Abordado:
1. Importar Bibliotecas Necessárias
2. Carregar e Explorar os Dados
3. Análise Exploratória
4. Visualização dos Dados
5. Limpeza dos Dados
6. Escolha do Melhor Modelo


## 1) Importar Bibliotecas Necessárias
Primeiramente, como sempre, importaremos bibliotecas Python como pandas, numpy, matplotlib e seaborn. Esta última biblioteca permite plotar uns gráficos diferentes, e mais bonitos se compararmos com a biblioteca matplotlib.

In [1]:
# bibliotecas para análise de dados
import numpy as np
import pandas as pd

# bibliotecas para visualização
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# configuração para ignorar mensagens de advertência (warning). 
# Muitas vezes estas mensagens atrapalham, pois pode trazer a informação de 
# algo está muito errado com nosso código quando na verdade temos apenas uma situação de advertência. 
import warnings
warnings.filterwarnings('ignore')

## 2) Carregar e Explorar os Dados
Cabe lembrar que a pergunta que devemos responder para este desafio é a seguinte: **Quais grupos de pessoas tiveram mais chances de sobreviver neste desastre? Para tanto, você deve explorar os dados de passageiros (ex. nome, idade, gênero, classe sócio-econômica, e outros).** 

O *dataset* de passageiros foi particionado em outros dois *datasets*: dados de treinamento e dados de teste. 
O *dataset* de treinamento contém os dados que serão utilizados para criar o nosso modelo preditivo, ou seja, os dados utilizados para **treinar o nosso modelo**. Por outro lado, o *dataset* de teste contém os dados que serão utilizados para testar e validar o desempenho do modelo com análise de sua acurácia e outras métricas. 

Neste passo você carregará os dois conjuntos de dados: treinamento e teste. Carregar neste contexto significa "importar" os dados dentro do nosso *notebook* utilizando o método `pd.read_csv`. Uma primeira olhada nos dados pode ser feita usando o método `describe()`.

In [8]:
# importar arquivos CSV: treinamento e teste. Note que os arquivos estão no Google Drive, e com isto, temos que acessá-los 
# através de um endereço web (URL) com caracteres 'estranhos' (chamados de 'hash'). Se os arquivos estivessem em seu computador,
# o acesso seria utilizando os nomes dos arquivos CSV treinamento.csv e teste.csv
url_arquivo_treinamento_csv='https://drive.google.com/uc?export=view&id=1dJWrDlgp43l9qcZtSJ8ArZSWvxKWaQCx'
url_arquivo_teste_csv='https://drive.google.com/uc?export=view&id=1wrYVj6EThqc1qX3QO_RWONGEQOf7tnJF'

treinamento_df= pd.read_csv(url_arquivo_treinamento_csv)
teste_df = pd.read_csv(url_arquivo_teste_csv)

# Dê uma olhada nos dados com o método describe. Passamos como parâmetro include=all para que todas as colunas do dataframe treinamento sejam apresentadas. 
treinamento_df.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"LeRoy, Miss. Bertha",male,NaN,NaN,NaN,1601,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [7]:
treinamento_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 3) Análise Exploratória
Agora, vamos analisar as variáveis (colunas) do *dataset* para verificar o tipo de dado, se dado faltante, se tem algum outro tipo de problema. Aqui resgatamos alguns dos conceitos de preparação de dados explorados nas semanas anteriores.
**Importante:** as variáveis (ou colunas) que utilizamos na criação da nossa solução, na forma de um modelo preditivo capaz de responder à nossa pergunta, recebem um nome especial em inglês, chamado de **features**. Uma feature é então uma variável que será utilizada como entrada na criação de um modelo preditivo.  

In [11]:
print(treinamento_df.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [12]:
treinamento_df.columns.values

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object)

In [ ]:
# mostramos uma amostra do dataframe para termos uma idéia das variáveis. 
# o método 'sample' nos permite pegar neste caso uma amostra com 5 linhas
treinamento_df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
77,78,0,3,"Moutal, Mr. Rahamin Haim",male,NaN,0,0,374746,8.0500,NaN,S
874,875,1,2,"Abelson, Mrs. Samuel (Hannah Wizosky)",female,28.00,1,0,P/PP 3381,24.0000,NaN,C
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
803,804,1,3,"Thomas, Master. Assad Alexander",male,0.42,0,1,2625,8.5167,NaN,C
105,106,0,3,"Mionoff, Mr. Stoytcho",male,28.00,0,0,349207,7.8958,NaN,S
